In [1]:
# @title Environment Setup
import os
import sys
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

MPLBACKEND environment variable cleared.
Cloning into 'FontDiffusion'...
remote: Enumerating objects: 20234, done.
remote: Counting objects: 100% (5046/5046), done.
remote: Compressing objects: 100% (4988/4988), done.
remote: Total 20234 (delta 126), reused 4927 (delta 56), pack-reused 15188 (from 3)
Receiving objects: 100% (20234/20234), 277.39 MiB | 38.77 MiB/s, done.
Resolving deltas: 100% (689/689), done.
Updating files: 100% (137/137), done.


In [2]:
import os
from IPython import get_ipython
from typing import Optional

def configure_environment_paths():
    """Detect environment and configure paths"""
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"
    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")
    return base_data_path, base_output_path, environment_name
def load_secret(key_name: str) -> Optional[str]:
    """
    Loads a secret key from the appropriate environment (Colab, Kaggle, or local env vars).
    Args:
        key_name (str): The name of the secret key to load (e.g., "WANDB_API_KEY", "HF_TOKEN").
    Returns:
        Optional[str]: The secret key value if found, otherwise None.
    """
    env = ENV_NAME
    secret_value = None
    print(f"Attempting to load secret '{key_name}' from '{env}' environment...")
    try:
        if env == "colab":
            from google.colab import userdata
            secret_value = userdata.get(key_name)
        elif env == "kaggle":
            from kaggle_secrets import UserSecretsClient
            user_secrets = UserSecretsClient()
            secret_value = user_secrets.get_secret(key_name)
        else: # Local environment
            secret_value = os.getenv(key_name)
        if not secret_value:
            print(f"⚠️ Secret '{key_name}' not found in the {env} environment.")
            return None
        print(f"✅ Successfully loaded secret '{key_name}'.")
        return secret_value
    except Exception as e:
        print(f"❌ An error occurred while loading secret '{key_name}': {e}")
        return None
INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()

✅ Environment: Kaggle
📂 Data Path: /kaggle/input/
📦 Output Path: /kaggle/working/


In [3]:
!uv pip install --upgrade pip
!uv pip install -r FontDiffusion/my_requirements.txt
# 3. Install PyTorch 1.13
%cd {OUTPUT_PATH}
# Force reinstall torch 1.13 to match the model's training environment
# !uv pip uninstall torch torchvision
# !uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!uv pip install torch torchvision
# 4. Install other dependencies
print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install lpips scikit-image pytorch-fid
!sudo apt-get update && sudo apt-get install dos2unix
!uv pip install gdown
!uv pip install wandb
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

Using Python 3.11.13 environment at: /usr
Resolved 1 package in 178ms                                          
Prepared 1 package in 126ms                                              
Uninstalled 1 package in 235ms
Installed 1 package in 18ms                                 
 - pip==24.1.2
 + pip==25.3
Using Python 3.11.13 environment at: /usr
  × No solution found when resolving dependencies:                                  
  ╰─▶ Because you require importlib-metadata==4.6.4 and
      importlib-metadata==8.0.0, we can conclude that your requirements are
      unsatisfiable.
/kaggle/working
Using Python 3.11.13 environment at: /usr
Resolved 38 packages in 112ms                                        
Uninstalled 10 packages in 45ms
Installed 10 packages in 19.30s.0.70                        
 - nvidia-cublas-cu12==12.5.3.2
 + nvidia-cublas-cu12==12.4.5.8
 - nvidia-cuda-cupti-cu12==12.5.82
 + nvidia-cuda-cupti-cu12==12.4.127
 - nvidia-cuda-nvrtc-cu12==12.5.82
 + nvidia-cuda-nvrtc-cu

In [4]:
# KAGGLE CELL #1: Download checkpoint

import os
import sys
from pathlib import Path
os.chdir(OUTPUT_PATH)
# Download from Hub
if not os.path.exists("ckpt") or not list(Path("ckpt").glob("*.safetensors")):
    print("📥 Downloading checkpoint from Hugging Face Hub...\n")
    from huggingface_hub import snapshot_download
    snapshot_download(
        repo_id="dzungpham/font-diffusion-weights",
        local_dir="ckpt",
        allow_patterns="*.safetensors",
        force_download=False
    )
    print("\n✅ Download complete!")
else:
    print("✅ Checkpoint already downloaded")
# Verify
print("\n📂 Files in ckpt/:")
for file in os.listdir("ckpt"):
    if file.endswith(".safetensors"):
        size = os.path.getsize(f"ckpt/{file}") / (1024**2)
        print(f"  ✓ {file} ({size:.2f} MB)")

📥 Downloading checkpoint from Hugging Face Hub...



Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

content_encoder.safetensors:   0%|          | 0.00/4.76M [00:00<?, ?B/s]

unet.safetensors:   0%|          | 0.00/315M [00:00<?, ?B/s]

style_encoder.safetensors:   0%|          | 0.00/82.4M [00:00<?, ?B/s]


✅ Download complete!

📂 Files in ckpt/:
  ✓ content_encoder.safetensors (4.54 MB)
  ✓ style_encoder.safetensors (78.58 MB)
  ✓ unet.safetensors (300.34 MB)


In [5]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

No .zip files found in /kaggle/input/.


In [6]:
import pandas as pd
import os

def convert_csv_to_chars_txt(input_csv_path: str, output_txt_path: str, column_name: str = 'word'):
    """
    Reads a CSV file, extracts text from a specified column, and writes each character
    to a new line in a plain text file.

    Args:
        input_csv_path (str): The full path to the input CSV file.
        output_txt_path (str): The full path for the output text file.
        column_name (str): The name of the column in the CSV file containing the text.
    """
    if not os.path.exists(input_csv_path):
        print(f"Error: Input CSV file not found at '{input_csv_path}'. Please ensure the file is uploaded.")
        return

    try:
        df = pd.read_csv(input_csv_path)
    except Exception as e:
        print(f"Error reading CSV file '{input_csv_path}': {e}")
        return

    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the CSV file '{input_csv_path}'.")
        return

    all_characters = []
    # Ensure the column values are treated as strings before iterating over them
    for item in df[column_name].astype(str).dropna().tolist():
        for char in item:
            all_characters.append(char)

    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)

    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(all_characters))
    print(f"Successfully converted '{input_csv_path}' to '{output_txt_path}', with one character per line.")

# --- Example Usage (demonstration with a dummy file) ---
# As the original file 'Ds_300_ChuNom_TuTao.csv' was not found in the previous execution,
# let's create a dummy file to demonstrate the function's usage.
print("\n--- Demonstrating function with a dummy CSV file ---")
dummy_csv_path = os.path.join(OUTPUT_PATH, "dummy_data.csv")
dummy_output_txt_path = os.path.join(OUTPUT_PATH, "dummy_chars.txt")

# Create a dummy CSV file
dummy_data = {'word': ['hello', 'world', 'python']}
pd.DataFrame(dummy_data).to_csv(dummy_csv_path, index=False)
print(f"Created a dummy CSV file at: {dummy_csv_path}")

convert_csv_to_chars_txt(dummy_csv_path, dummy_output_txt_path)

# --- How to use with your actual file ---
# Uncomment the lines below and replace 'your_actual_file.csv' and 'your_output.txt'
# with the correct paths for your use case.
#
# original_csv_file = os.path.join(INPUT_PATH, "Ds_300_ChuNom_TuTao.csv") # Or the full path to your CSV
# original_output_txt = os.path.join(OUTPUT_PATH, "nom_tu_tao.txt") # Or your desired output path
# convert_csv_to_chars_txt(original_csv_file, original_output_txt)



--- Demonstrating function with a dummy CSV file ---
Created a dummy CSV file at: /kaggle/working/dummy_data.csv
Successfully converted '/kaggle/working/dummy_data.csv' to '/kaggle/working/dummy_chars.txt', with one character per line.


In [7]:
!ls -larth {OUTPUT_PATH}/ckpt

total 384M
drwxr-xr-x 3 root root 4.0K Dec 31 10:34 .cache
-rw-r--r-- 1 root root 4.6M Dec 31 10:35 content_encoder.safetensors
-rw-r--r-- 1 root root  79M Dec 31 10:35 style_encoder.safetensors
-rw-r--r-- 1 root root 301M Dec 31 10:35 unet.safetensors
drwxr-xr-x 3 root root 4.0K Dec 31 10:35 .
drwxr-xr-x 5 root root 4.0K Dec 31 10:35 ..


In [8]:
%cd {OUTPUT_PATH}
from huggingface_hub import login
HF_TOKEN = load_secret("HF_TOKEN")
login(HF_TOKEN)
HF_USERNAME = "dzungpham"

# ==========================================
# EXPORT / DOWNLOAD DATASET COMMANDS
# ==========================================

# Train Split
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train" \
  --token HF_TOKEN 
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train_original" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train_original" \
  --token HF_TOKEN 
# Validation: Unseen Both
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val_unseen_both" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val_unseen_both" \
  --token HF_TOKEN 
# Validation: Seen Style, Unseen Char
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val_seen_style_unseen_char" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val_seen_style_unseen_char" \
  --token HF_TOKEN 
# Validation: Unseen Style, Seen Char
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val_unseen_style_seen_char" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val_unseen_style_seen_char" \
  --token HF_TOKEN 
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/test" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "test" \
  --token HF_TOKEN 
print("SUCCESSFULLY EXPORT HF DATASET TO LOCAL DIRECTORY")

/kaggle/working
Attempting to load secret 'HF_TOKEN' from 'kaggle' environment...
✅ Successfully loaded secret 'HF_TOKEN'.

EXPORTING DATASET TO DISK

📥 Loading dataset from Hub...
   Repository: dzungpham/font-diffusion-generated-data
   Split: train
README.md: 1.29kB [00:00, 804kB/s]
data/train-00000-of-00001.parquet: 100%|███| 39.5M/39.5M [00:01<00:00, 32.5MB/s]
data/val-00000-of-00001.parquet: 100%|███████| 262k/262k [00:00<00:00, 1.27MB/s]
data/val_unseen_both-00000-of-00001.parq(…): 100%|█| 2.52M/2.52M [00:00<00:00, 1
data/val_seen_style_unseen_char-00000-of(…): 100%|█| 8.02M/8.02M [00:00<00:00, 1
data/val_unseen_style_seen_char-00000-of(…): 100%|█| 10.5M/10.5M [00:00<00:00, 3
data/train_original-00000-of-00001.parqu(…): 100%|█| 68.8M/68.8M [00:00<00:00, 2
data/test-00000-of-00001.parquet: 100%|██████| 646k/646k [00:00<00:00, 4.37MB/s]
Generating val split: 100%|████████████| 24/24 [00:00<00:00, 7095.96 examples/s]
Generating val_unseen_both split: 100%|█| 240/240 [00:00<00:00, 2

In [9]:
!uv pip install "huggingface-hub==0.25.2"

Using Python 3.11.13 environment at: /usr
Resolved 12 packages in 57ms                                         
Prepared 1 package in 58ms                                               
Uninstalled 1 package in 12ms
Installed 1 package in 5ms25.2                              
 - huggingface-hub==0.36.0
 + huggingface-hub==0.25.2


In [ ]:
# already change sample_batch file to save all data in train_original
%cd {OUTPUT_PATH}
!python FontDiffusion/sample_batch.py \
    --characters "FontDiffusion/NomTuTao/Ds_10k_ChuNom_TuTao.txt" \
    --style_images "FontDiffusion/styles_images" \
    --ckpt_dir "ckpt/" \
    --ttf_path "FontDiffusion/fonts/NomNaTong-Regular.otf" \
    --output_dir "my_dataset/train_original" \
    --resume_from "my_dataset/train_original/results_checkpoint.json" \
    --num_inference_steps 20 \
    --guidance_scale 7.5 \
    --start_line 1 \
    --end_line 50 \
    --batch_size 24 \
    --save_interval 1 \
    --channels_last \
    --seed 42 \
    --compile \
    --enable_xformers

/kaggle/working
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
2025-12-31 10:36:32.715306: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1

In [ ]:
!python FontDiffusion/create_validation_split.py \
  --data_root my_dataset \
  --val_ratio 0.2 \
  --test_ratio 0.1 \
  --seed 42

In [ ]:
# --- RAW DATA (Before Splitting) ---
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train_original" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "train_original" \
  --private \
  --token "{HF_TOKEN}"

# --- ORGANIZED SPLITS (After Splitting) ---

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "train" \
  --private \
  --token "{HF_TOKEN}"

# Test Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/test" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "test" \
  --private \
  --token "{HF_TOKEN}"

# Validation: Unseen Both
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val_unseen_both" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "val_unseen_both" \
  --private \
  --token "{HF_TOKEN}"

# Validation: Seen Style, Unseen Char
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val_seen_style_unseen_char" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "val_seen_style_unseen_char" \
  --private \
  --token "{HF_TOKEN}"

# Validation: Unseen Style, Seen Char
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val_unseen_style_seen_char" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "val_unseen_style_seen_char" \
  --private \
  --token "{HF_TOKEN}"
print("SUCCESSFULLY UPLOAD LOCAL MY_DATASET TO HUGGINGFACE DATASETS SPACE")

In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# TRAINING PHASE 1
import wandb

# Load your Weights & Biases API key from a secure store
wandb_key = load_secret("WANDB_API_KEY")
wandb.login(key=wandb_key)

# Run the training script with the corrected flag syntax
!python FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name=FontDiffuser_training_phase_1 \
    --data_root=my_dataset \
    --output_dir=outputs/FontDiffuser \
    --report_to=wandb \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.7 \
    --max_train_steps=2000 \
    --ckpt_interval=1000 \
    --val_interval=200 \
    --gradient_accumulation_steps=1 \
    --log_interval=50 \
    --learning_rate=1e-4 \
    --lr_scheduler=linear \
    --lr_warmup_steps=10000 \
    --drop_prob=0.1 \
    --mixed_precision=no

In [ ]:
!ls -lr outputs/FontDiffuser

In [ ]:
# TRAINING PHASE 2
!wandb login
!python FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_2" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    --phase_2 \
    --phase_1_ckpt_dir="outputs/FontDiffuser/global_step_2000" \
    --scr_ckpt_path="ckpt/scr_210000.pth" \
    --sc_coefficient=0.05 \
    --num_neg=13 \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.4 \
    --max_train_steps=100 \
    --ckpt_interval=50 \
    --gradient_accumulation_steps=2 \
    --log_interval=50 \
    --learning_rate=1e-5 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=1000 \
    --drop_prob=0.1 \
    --mixed_precision="no"


In [ ]:
!python FontDiffusion/pth2safetensors.py \
    --weights_dir "ckpt" \
    --repo_id "dzungpham/font-diffusion-weights" \
    --token "{HF_TOKEN}"

In [ ]:
import os
import zipfile
from pathlib import Path
from typing import List
def find_result_folders(base_path: Path, pattern_name: str) -> List[Path]:
    return [p for p in base_path.glob(pattern_name) if p.is_dir()]

def zip_folder(folder_path: Path, output_base_path: Path) -> bool:
    folder_name = folder_path.name
    zip_path = output_base_path / f"{folder_name}.zip"
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zipf:
            for file_path in folder_path.rglob("*"):
                if file_path.is_file():
                    arcname = file_path.relative_to(folder_path.parent)
                    zipf.write(file_path, arcname)
        print(f"   ✅ Created ZIP: {zip_path.name}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False

def zip_stats_results_folders(output_base_path: str, pattern_name: str) -> None:
    base = Path(output_base_path)
    base.mkdir(parents=True, exist_ok=True)
    result_folders = find_result_folders(base, pattern_name)
    if not result_folders:
        print(f"⚠️ No folders matching '*dataset' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = sum(1 for folder in result_folders if zip_folder(folder, base))
    print(f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s).")

if __name__ == "__main__":
    try:
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        zip_stats_results_folders(
            output_base_path=OUTPUT_PATH,
            pattern_name="outputs/FontDiffuser/global*")
    except Exception as e:
        print(f"❌ An error occurred: {e}")